In [1]:
import os
print(os.environ['CONDA_DEFAULT_ENV'])
#.\venv_mediapipe_protobuf\Scripts\activate    to activate virtual env 

opencv-env


In [2]:
#pip install numpy~=1.26.4 scikit-learn~=1.5.0 tensorflow~=2.16.1 opencv-python~=4.10.0.82 mediapipe~=0.10.14 pandas~=2.2.2 pytube~=15.0.0

In [3]:
#!pip install pytube


In [4]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import mediapipe as mp
import time
from pytube import YouTube



In [5]:
# Extracting feature keypoints using MediaPipe holistic model
mp_holistic = mp.solutions.holistic  # Holistic model
mp_drawing_styles = mp.solutions.drawing_styles
mp_drawing = mp.solutions.drawing_utils  # Drawing utilities


In [6]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert image from BGR to RGB
    image.flags.writeable = False  # Set the 'writable' flag to False to improve performance
    results = model.process(image)  # Make predictions
    image.flags.writeable = True  # Set the 'writable' flag back to True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert image back to BGR
    return image, results



In [17]:
def draw_styled_landmarks(image, results):
    # Define drawing specs for each landmark type
    face_spec = mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=1, circle_radius=1)  # red, thickness 1, circle radius 1
    pose_spec = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)  # green, thickness 2, circle radius 2
    hand_spec = mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)  # blue, thickness 2, circle radius 2

    # Draw styled landmarks
    if results.face_landmarks:
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, face_spec)
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, pose_spec)
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, hand_spec)
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, hand_spec)

    

   

In [18]:
def capture_frames(folder_name, frame, count):
    # Create folder if it doesn't exist
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    # Save the captured frame
    frame_path = os.path.join(folder_name, f"frame_{count}.png")
    cv2.imwrite(frame_path, frame)
    print(f"Captured {frame_path}")

In [19]:
def primary_data_capture():
    holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5, enable_segmentation=True)
    cap = cv2.VideoCapture(0)
    window_name = "Holistic Model Detection"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)

    folder_name = input("Enter the name of the folder to save the frames: ")
    subfolder_name = input("Enter the name of the subfolder (e.g., an alphabet): ")
    full_path = os.path.join(folder_name, subfolder_name)

    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image)

        # Draw styled landmarks
        draw_styled_landmarks(frame, results)

        # Display the frame
        cv2.imshow(window_name, frame)

        # Display the segmentation mask
        if results.segmentation_mask is not None and results.segmentation_mask.shape[0] > 0 and results.segmentation_mask.shape[1] > 0:
            segmentation_mask_frame = cv2.flip(results.segmentation_mask, 1)
            cv2.imshow("Segmentation Mask", segmentation_mask_frame)
        

        key = cv2.waitKey(1) & 0xFF
        if key == ord('c'):
            frame_count += 1
            capture_frames(full_path, frame, frame_count)
        elif key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [20]:
def secondary_data_capture(video_path):
    holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5, enable_segmentation=True)
    cap = cv2.VideoCapture(video_path)
    
    window_name = "Holistic Model Detection "
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)

    folder_name = input("Enter the name of the folder to save the frames: ")
    subfolder_name = input("Enter the name of the subfolder (e.g., an alphabet): ")
    full_path = os.path.join(folder_name, subfolder_name)

    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image)

        # Draw styled landmarks
        draw_styled_landmarks(frame, results)

        # Display the frame
        cv2.imshow(window_name, frame)

        # Display the segmentation mask
        if results.segmentation_mask is not None and results.segmentation_mask.shape[0] > 0 and results.segmentation_mask.shape[1] > 0:
            segmentation_mask_frame = cv2.flip(results.segmentation_mask, 1)
            cv2.imshow("Segmentation Mask", segmentation_mask_frame)
       
        key = cv2.waitKey(1) & 0xFF
        if key == ord('c'):
            frame_count += 1
            capture_frames(full_path, frame, frame_count)
        elif key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [16]:
def main():
    print("Choose data capture method:")
    print("Press 'p' for Primary Data Capture (camera)")
    print("Press 's' for Secondary Data Capture (YouTube video)")

    cv2.namedWindow("Choose Data Capture Method", cv2.WINDOW_NORMAL)
    cv2.imshow("Choose Data Capture Method", np.zeros((100, 200, 3), dtype=np.uint8))

    while True:
        key = cv2.waitKey(1) & 0xFF
        if key == ord('p'):
            primary_data_capture()
            break
        elif key == ord('s'):
            video_path = input("Enter video path: ")
            secondary_data_capture(video_path)
            break

    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Choose data capture method:
Press 'p' for Primary Data Capture (camera)
Press 's' for Secondary Data Capture (YouTube video)


Enter video path:  D:\1School\project\ksl videos\trimmed\KSL 1 2023 Topic 1   Alphabets (2).mp4
Enter the name of the folder to save the frames:  Alphabets
Enter the name of the subfolder (e.g., an alphabet):  Aa


AttributeError: 'SymbolDatabase' object has no attribute 'GetPrototype'